In [1]:
import pandas as pd

# S3 path to the uploaded diabetes.csv file
s3_uri = 's3://diabetes-predict-ml-data/diabetes.csv'

# Load CSV into a DataFrame
df = pd.read_csv(s3_uri)
df.head()


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:286: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [2]:
!pip install sagemaker --upgrade

import sagemaker
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput
from sagemaker.estimator import Estimator
from sklearn.model_selection import train_test_split



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 74.4 MB/s eta 0:00:00
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.243.3
    Uninstalling sagemaker-2.243.3:
      Successfully uninstalled sagemaker-2.243.3


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[05/26/25 07:35:19] INFO     Found credentials from IAM Role:                                   ]8;id=689488;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=872575;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
from sagemaker.image_uris import retrieve

# Get the XGBoost image URI for your region
container = retrieve("xgboost", sagemaker.Session().boto_region_name, version="1.2-2")

# Define the estimator (training job)
xgb_estimator = Estimator(
    image_uri=container,
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type="ml.m5.large",
    output_path=f"s3://{bucket}/xgboost/output",
    sagemaker_session=session
)

# Set hyperparameters
xgb_estimator.set_hyperparameters(
    objective="binary:logistic",
    num_round=100
)

# Input format (from previously uploaded data)
train_input = TrainingInput(train_s3_path, content_type="csv")

# Launch training job
xgb_estimator.fit({"train": train_input})


[05/26/25 07:38:52] INFO     Found credentials from IAM Role:                                   ]8;id=33654;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=949627;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[05/26/25 07:38:53] INFO     Ignoring unnecessary instance type: None.                            ]8;id=188672;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=384491;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#530\530]8;;\

                    INFO     Found credentials from IAM Role:                                   ]8;id=806142;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=849216;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:12                                                                                   │
│                                                                                                  │
│    9 │   role=sagemaker.get_execution_role(),                                                    │
│   10 │   instance_count=1,                                                                       │
│   11 │   instance_type="ml.m5.large",                                                            │
│ ❱ 12 │   output_path=f"s3://{bucket}/xgboost/output",                                            │
│   13 │   sagemaker_session=session                                                               │
│   14 )                                                                                           │
│   15                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'bucket' is not defined

In [4]:
from sagemaker.image_uris import retrieve

# Define your bucket
bucket = "diabetes-predict-ml-data"  # 👈 Your bucket name from S3

# Get XGBoost container
container = retrieve("xgboost", sagemaker.Session().boto_region_name, version="1.2-2")

# Create estimator
xgb_estimator = Estimator(
    image_uri=container,
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type="ml.m5.large",
    output_path=f"s3://{bucket}/xgboost/output",  # 👈 Now this will work
    sagemaker_session=session
)

xgb_estimator.set_hyperparameters(
    objective="binary:logistic",
    num_round=100
)

# Train input from earlier
train_input = TrainingInput(train_s3_path, content_type="csv")

# Start training
xgb_estimator.fit({"train": train_input})



[05/26/25 07:40:48] INFO     Found credentials from IAM Role:                                   ]8;id=579972;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=447124;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Ignoring unnecessary instance type: None.                            ]8;id=892557;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=944163;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#530\530]8;;\

                    INFO     Found credentials from IAM Role:                                   ]8;id=188713;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=492918;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:16                                                                                   │
│                                                                                                  │
│   13 │   instance_count=1,                                                                       │
│   14 │   instance_type="ml.m5.large",                                                            │
│   15 │   output_path=f"s3://{bucket}/xgboost/output",  # 👈 Now this will work                   │
│ ❱ 16 │   sagemaker_session=session                                                               │
│   17 )                                                                                           │
│   18                                                                                             │
│   19 xgb_estimator.set_hyperparameters(                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'session' is not defined

In [5]:
import sagemaker
from sagemaker.image_uris import retrieve
from sagemaker.estimator import Estimator

# Define session and bucket
session = sagemaker.Session()
bucket = "diabetes-predict-ml-data"  # <-- your actual S3 bucket name

# Get container
container = retrieve("xgboost", session.boto_region_name, version="1.2-2")

# Create estimator
xgb_estimator = Estimator(
    image_uri=container,
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type="ml.m5.large",
    output_path=f"s3://{bucket}/xgboost/output",
    sagemaker_session=session
)

# Set training hyperparameters
xgb_estimator.set_hyperparameters(
    objective="binary:logistic",
    num_round=100
)

# Create train input
from sagemaker.inputs import TrainingInput
train_input = TrainingInput(train_s3_path, content_type="csv")

# Fit model (this will start training)
xgb_estimator.fit({"train": train_input})


[05/26/25 07:42:18] INFO     Found credentials from IAM Role:                                   ]8;id=503995;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=372766;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Ignoring unnecessary instance type: None.                            ]8;id=877468;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=919048;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#530\530]8;;\

                    INFO     Found credentials from IAM Role:                                   ]8;id=146256;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=694880;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[05/26/25 07:42:19] INFO     Found credentials from IAM Role:                                   ]8;id=746983;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=533202;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:30                                                                                   │
│                                                                                                  │
│   27                                                                                             │
│   28 # Create train input                                                                        │
│   29 from sagemaker.inputs import TrainingInput                                                  │
│ ❱ 30 train_input = TrainingInput(train_s3_path, content_type="csv")                              │
│   31                                                                                             │
│   32 # Fit model (this will start training)                                                      │
│   33 xgb_estimator.fit({"train": train_input})                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'train_s3_path' is not defined

In [6]:
# Upload training and test data to S3
train_s3_path = session.upload_data('train.csv', bucket=bucket, key_prefix='xgboost')
test_s3_path = session.upload_data('test.csv', bucket=bucket, key_prefix='xgboost')

print("Train S3 Path:", train_s3_path)
print("Test S3 Path:", test_s3_path)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # Upload training and test data to S3                                                        │
│ ❱ 2 train_s3_path = session.upload_data('train.csv', bucket=bucket, key_prefix='xgboost')        │
│   3 test_s3_path = session.upload_data('test.csv', bucket=bucket, key_prefix='xgboost')          │
│   4                                                                                              │
│   5 print("Train S3 Path:", train_s3_path)                                                       │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py:432 in   │
│ upload_data                                                                                      │
│                                                                                                  │
│    429 │   │   │   s3 = self.s3_resource                                                         │
│    430 │   │                                                                                     │
│    431 │   │   for local_path, s3_key in files:                                                  │
│ ❱  432 │   │   │   s3.Object(bucket, s3_key).upload_file(                                        │
│    433 │   │   │   │   local_path, Callback=callback, ExtraArgs=extra_args                       │
│    434 │   │   │   )                                                                             │
│    435                                                                                           │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/boto3/s3/inject.py:351 in     │
│ object_upload_file                                                                               │
│                                                                                                  │
│   348 │   :param Config: The transfer configuration to be used when performing the               │
│   349 │   │   transfer.                                                                          │
│   350 │   """                                                                                    │
│ ❱ 351 │   return self.meta.client.upload_file(                                                   │
│   352 │   │   Filename=Filename,                                                                 │
│   353 │   │   Bucket=self.bucket_name,                                                           │
│   354 │   │   Key=self.key,                                                                      │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/context.py:123 in    │
│ wrapper                                                                                          │
│                                                                                                  │
│   120 │   │   │   with start_as_current_context():                                               │
│   121 │   │   │   │   if hook:                                                                   │
│   122 │   │   │   │   │   hook()                                                                 │
│ ❱ 123 │   │   │   │   return func(*args, **kwargs)                                               │
│   124 │   │                                                                                      │
│   125 │   │   return wrapper                               

In [7]:
# Recreate train.csv and test.csv locally

# Clean nulls (if any)
df = df.dropna()

# Split features and label
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Add labels back for SageMaker
X_train['Outcome'] = y_train
X_test['Outcome'] = y_test

# Save files locally
X_train.to_csv('train.csv', index=False, header=False)
X_test.to_csv('test.csv', index=False, header=False)

print("✅ train.csv and test.csv created")


✅ train.csv and test.csv created


In [8]:
train_s3_path = session.upload_data('train.csv', bucket=bucket, key_prefix='xgboost')
test_s3_path = session.upload_data('test.csv', bucket=bucket, key_prefix='xgboost')

print("Train S3 Path:", train_s3_path)
print("Test S3 Path:", test_s3_path)


Train S3 Path: s3://diabetes-predict-ml-data/xgboost/train.csv
Test S3 Path: s3://diabetes-predict-ml-data/xgboost/test.csv


In [9]:
from sagemaker.image_uris import retrieve
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput

# Retrieve XGBoost image
container = retrieve("xgboost", sagemaker.Session().boto_region_name, version="1.2-2")

# Define estimator
xgb_estimator = Estimator(
    image_uri=container,
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type="ml.m5.large",
    output_path=f"s3://{bucket}/xgboost/output",
    sagemaker_session=session
)

# Set hyperparameters
xgb_estimator.set_hyperparameters(
    objective="binary:logistic",
    num_round=100
)

# Training input reference
train_input = TrainingInput(train_s3_path, content_type="csv")

# Start training
xgb_estimator.fit({"train": train_input})


[05/26/25 07:47:12] INFO     Ignoring unnecessary instance type: None.                            ]8;id=369288;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=262323;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#530\530]8;;\

                    INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=799187;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=532047;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=458431;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=564013;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1053\1053]8;;\
                             sagemaker-xgboost-2025-05-26-07-47-12-953                                             

2025-05-26 07:47:15 Starting - Starting the training job...
2025-05-26 07:47:30 Starting - Preparing the instances for training...
2025-05-26 07:47:50 Downloading - Downloading input data...
2025-05-26 07:48:30 Downloading - Downloading the training image.....
2025-05-26 07:49:37 Training - Training image download completed. Training in progress.
2025-05-26 07:49:37 Uploading - Uploading generated training model[2025-05-26 07:49:30.761 ip-10-0-219-198.eu-north-1.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-05-26:07:49:30:INFO] Imported framework sagemaker_xgboost_container.training
[2025-05-26:07:49:30:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2025-05-26:07:49:30:INFO] No GPUs detected (normal if no gpus installed)
[2025-05-26:07:49:30:INFO] Running XGBoost Sagemaker in algorithm mode
[2025-05-26:07:49:30:INFO] Determined delimiter of CSV input is ','
[2025-05-26:07:49:30:INFO] Determined 

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:28                                                                                   │
│                                                                                                  │
│   25 train_input = TrainingInput(train_s3_path, content_type="csv")                              │
│   26                                                                                             │
│   27 # Start training                                                                            │
│ ❱ 28 xgb_estimator.fit({"train": train_input})                                                   │
│   29                                                                                             │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry │
│ _logging.py:167 in wrapper                                                                       │
│                                                                                                  │
│   164 │   │   │   │   │   caught_ex = e                                                          │
│   165 │   │   │   │   finally:                                                                   │
│   166 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 167 │   │   │   │   │   │   raise caught_ex                                                    │
│   168 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   169 │   │   │   else:                                                                          │
│   170 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry │
│ _logging.py:138 in wrapper                                                                       │
│                                                                                                  │
│   135 │   │   │   │   start_timer = perf_counter()                                               │
│   136 │   │   │   │   try:                                                                       │
│   137 │   │   │   │   │   # Call the original function                                           │
│ ❱ 138 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   139 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   140 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   141 │   │   │   │   │   extra += f"&x-latency={round(elapsed, 2)}"                             │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/workflow/pipeline_c │
│ ontext.py:346 in wrapper                                                                         │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                   

In [10]:

# Label as first column
train_data = pd.concat([y_train, X_train], axis=1)
test_data = pd.concat([y_test, X_test], axis=1)

# Save for SageMaker (no headers, no index)
train_data.to_csv("train.csv", index=False, header=False)
test_data.to_csv("test.csv", index=False, header=False)

print("✅ Reformatted: label first, files saved again.")


✅ Reformatted: label first, files saved again.


In [11]:
train_s3_path = session.upload_data('train.csv', bucket=bucket, key_prefix='xgboost')
test_s3_path = session.upload_data('test.csv', bucket=bucket, key_prefix='xgboost')

print("✅ Upload complete")
print(train_s3_path)


✅ Upload complete
s3://diabetes-predict-ml-data/xgboost/train.csv


In [12]:
from sagemaker.inputs import TrainingInput
train_input = TrainingInput(train_s3_path, content_type="csv")

xgb_estimator.fit({"train": train_input})


[05/26/25 08:00:01] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=893076;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=756961;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=450966;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=368253;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1053\1053]8;;\
                             sagemaker-xgboost-2025-05-26-08-00-01-433                                             

2025-05-26 08:00:03 Starting - Starting the training job...
2025-05-26 08:00:35 Downloading - Downloading input data......
2025-05-26 08:01:16 Downloading - Downloading the training image.....[2025-05-26 08:02:16.550 ip-10-0-127-205.eu-north-1.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-05-26:08:02:16:INFO] Imported framework sagemaker_xgboost_container.training
[2025-05-26:08:02:16:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2025-05-26:08:02:16:INFO] No GPUs detected (normal if no gpus installed)
[2025-05-26:08:02:16:INFO] Running XGBoost Sagemaker in algorithm mode
[2025-05-26:08:02:16:INFO] Determined delimiter of CSV input is ','
[2025-05-26:08:02:16:INFO] Determined delimiter of CSV input is ','
[2025-05-26:08:02:16:INFO] Single node training.
[2025-05-26:08:02:16:INFO] Train matrix has 614 rows and 9 columns
[2025-05-26 08:02:16.615 ip-10-0-127-205.eu-north-1.compute.internal:7 INFO j

In [ ]:
# Deploy the trained model as an endpoint
predictor = xgb_estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large"
)


[05/26/25 08:06:18] INFO     Creating model with name: sagemaker-xgboost-2025-05-26-08-06-18-092    ]8;id=932152;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=210007;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4105\4105]8;;\

                    INFO     Creating endpoint-config with name                                     ]8;id=566206;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=942847;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#6030\6030]8;;\
                             sagemaker-xgboost-2025-05-26-08-06-18-092                                             

[05/26/25 08:06:19] INFO     Creating endpoint with name sagemaker-xgboost-2025-05-26-08-06-18-092  ]8;id=850607;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=6497;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4852\4852]8;;\

---

In [14]:
import numpy as np

# Sample patient data (in same order as training features):
# Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DPF, Age
sample_input = np.array([[6, 148, 72, 35, 0, 33.6, 0.627, 50]])

# Predict
prediction = predictor.predict(sample_input)
print("Prediction:", prediction)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:8                                                                                    │
│                                                                                                  │
│    5 sample_input = np.array([[6, 148, 72, 35, 0, 33.6, 0.627, 50]])                             │
│    6                                                                                             │
│    7 # Predict                                                                                   │
│ ❱  8 prediction = predictor.predict(sample_input)                                                │
│    9 print("Prediction:", prediction)                                                            │
│   10                                                                                             │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/base_predictor.py:2 │
│ 12 in predict                                                                                    │
│                                                                                                  │
│   209 │   │   if inference_component_name:                                                       │
│   210 │   │   │   request_args["InferenceComponentName"] = inference_component_name              │
│   211 │   │                                                                                      │
│ ❱ 212 │   │   response = self.sagemaker_session.sagemaker_runtime_client.invoke_endpoint(**req   │
│   213 │   │   return self._handle_response(response)                                             │
│   214 │                                                                                          │
│   215 │   def _handle_response(self, response):                                                  │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/client.py:570 in     │
│ _api_call                                                                                        │
│                                                                                                  │
│    567 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    568 │   │   │   │   )                                                                         │
│    569 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  570 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    571 │   │                                                                                     │
│    572 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    573                                                                                           │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/context.py:123 in    │
│ wrapper                                                                                          │
│                                                                                                  │
│   120 │   │   │   with start_as_current_context():                                               │
│   121 │   │   │   │   if hook:                                                                   │
│   122 │   │   │   │   │   hook()                                                                 │
│ ❱ 123 │   │   │   │   return func(*args, **kwargs)                                               │
│   124 │   │                                                

In [15]:
import numpy as np
import json

# Sample input (match training feature order)
sample_input = [[6, 148, 72, 35, 0, 33.6, 0.627, 50]]

# Convert to CSV string format as expected by SageMaker XGBoost
csv_input = ",".join(map(str, sample_input[0]))

# Predict (content_type="text/csv" is key!)
result = predictor.predict(csv_input, initial_args={"ContentType": "text/csv"})

print("✅ Prediction Result:", result)


✅ Prediction Result: b'0.0030567164067178965\n'
